# 特徴点マッチングによる物体検出

Opencvのインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q opencv-python-headless

In [ ]:
%run setup_demo_env.py

In [ ]:
import time

import cv2
import grpc
import kachaka_api_pb2
import numpy as np
from IPython.display import Image, clear_output, display
from kachaka_api_pb2_grpc import KachakaApiStub

In [ ]:
stub = KachakaApiStub(grpc.aio.insecure_channel(kachaka_api_server))

以下を実行したら、認識させたい物体をカメラ画像の赤枠内に表示させて下さい。10秒経つと撮影されます。

In [ ]:
req = kachaka_api_pb2.GetRequest()

target_area_length = 400
time_to_capture = 10

start_time = time.time()
while True:
    resp = await stub.GetFrontCameraRosCompressedImage(req)
    req.metadata.cursor = resp.metadata.cursor
    image = cv2.imdecode(np.frombuffer(resp.image.data, dtype=np.uint8), flags=1)
    start_x = int(image.shape[1] / 2 - target_area_length / 2)
    start_y = int(image.shape[0] / 2 - target_area_length / 2)
    end_x = int(image.shape[1] / 2 + target_area_length / 2)
    end_y = int(image.shape[0] / 2 + target_area_length / 2)

    remaining_time = time_to_capture - (time.time() - start_time)
    text = f"{remaining_time:.2f} [sec]" if remaining_time > 0 else "capture!"

    image = cv2.rectangle(
        image, (start_x, start_y), (end_x, end_y), (0, 0, 255), thickness=2
    )
    image = cv2.putText(
        image,
        text,
        (start_x, start_y - 10),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
        cv2.LINE_AA,
    )
    _, ret = cv2.imencode(
        ".jpg", cv2.resize(image, (int(image.shape[1] / 2), int(image.shape[0] / 2)))
    )
    clear_output(wait=True)
    display(Image(data=ret, format="jpeg"))
    if remaining_time <= 0.0:
        break
    req.metadata.cursor = resp.metadata.cursor

orig_image = cv2.imdecode(np.frombuffer(resp.image.data, dtype=np.uint8), flags=1)[
    start_y:end_y, start_x:end_x
]
orb = cv2.ORB_create()
orig_keypoints, orig_descriptors = orb.detectAndCompute(orig_image, None)

撮影した物体をカメラに映してみて下さい。特徴点マッチングが行われている様子を確認できます。

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING)
ratio_threshold = 0.75

while True:
    resp = await stub.GetFrontCameraRosCompressedImage(req)
    image = cv2.imdecode(np.frombuffer(resp.image.data, dtype=np.uint8), flags=1)
    keypoints, descriptors = orb.detectAndCompute(image, None)
    matches = bf.knnMatch(orig_descriptors, descriptors, k=2)
    good = []
    for m, n in matches:
        if m.distance < ratio_threshold * n.distance:
            good.append([m])
    image = cv2.drawMatchesKnn(
        orig_image,
        orig_keypoints,
        image,
        keypoints,
        good,
        None,
        flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS,
    )
    image = cv2.putText(
        image,
        f"good matches: {len(good)}",
        (50, image.shape[0] - 150),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 0, 255),
        2,
        cv2.LINE_AA,
    )
    clear_output(wait=True)
    _, ret = cv2.imencode(
        ".jpg", cv2.resize(image, (int(image.shape[1] / 2), int(image.shape[0] / 2)))
    )
    display(Image(data=ret, format="jpeg"))
    req.metadata.cursor = resp.metadata.cursor